# Command Hammer

A Command and Colours derivative for Warhammer Fantasy

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.signal as sig
import scipy.stats as stats
from fractions import Fraction
import matplotlib.pyplot as plt

In [2]:
sns.set_theme()

## Simulate    

In [3]:
def attack_roll(attacks, attacker_ws, defender_ws, samples=100000):
    target = min(max(3, 7 - (attacker_ws - defender_ws)), 12)
    rolls = stats.randint(1, 7).rvs((samples, 2)).sum(axis=1)
    successes = np.where(rolls >= target, rolls - target + 1, 0)
    bounded = np.where(successes >= attacks, attacks, successes)
    return bounded

In [4]:
def break_roll(hits, attacker_s, defender_t):
    target = min(max(3, 7 - (attacker_s - defender_t)), 12)
    rolls = stats.randint(1, 7).rvs((hits.size, 2)).sum(axis=1)
    successes = np.where((hits != 0) & (rolls >= target), rolls - target + 1, 0)
    damage = np.where(successes >= hits, hits, successes)
    retreats = np.where(successes > hits, successes - hits, 0)
    return np.concatenate([damage.reshape(-1, 1), retreats.reshape(-1, 1)], axis=1)

In [5]:
def adjusted_break(breaks, defender_ld):
    damage = breaks[:, 0]
    base = breaks[:, 1]
    if defender_ld >= 0:
        retreats = np.where(base > defender_ld, base - defender_ld, 0)
    else:
        abs_ld = (- defender_ld)
        retreats = np.where(damage > abs_ld, base + abs_ld, base + damage)
    return np.concatenate([damage.reshape(-1, 1), retreats.reshape(-1, 1)], axis=1)

In [6]:
def combat_resolution(attacks, attacker_ws, defender_ws, attacker_s, defender_t, defender_ld, samples=10000):
    return adjusted_break(break_roll(attack_roll(attacks, attacker_ws, defender_ws, samples=samples), 
                                     attacker_s, defender_t), 
                          defender_ld)

In [7]:
def combat_stats(resolution):
    df = (pd.DataFrame(resolution, columns=["damage", "retreats"])
          .assign(freq=1)
          .groupby(["damage", "retreats"])
          .count()
          .assign(prob=lambda df: df["freq"]/df["freq"].sum())
          .reset_index()
          .sort_values("prob", ascending=False))
    return df

In [8]:
def combat_expectation(resolution):
    return resolution.mean(axis=0)

In [9]:
print("In combat", combat_expectation(combat_resolution(4, 0, 0, 0, 0, 0)))
print("One Charger", combat_expectation(combat_resolution(4, 1, 0, 0, 0, -1)))
print("Two Chargers", combat_expectation(combat_resolution(4, 2, 0, 0, 0, -3)))
print("Three Chargers", combat_expectation(combat_resolution(4, 3, 0, 0, 0, -5)))

In combat [0.6191 0.3044]
One Charger [0.8313 0.7174]
Two Chargers [1.0313 1.2367]
Three Chargers [1.1814 1.4254]
